This Notebook will be used for the capstone project.

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [68]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import io

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


!pip install folium
import folium # map rendering library

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

print('done')

done


In [82]:
url = "https://raw.githubusercontent.com/degulor/Coursera_Capstone/main/QCMandaluyongCoordinates.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe

df = pd.read_csv(io.StringIO(download.decode('latin-1')))

df.sort_values(by=['Barangay'], inplace=True)

df = df.reset_index(drop=True)

# Printing out the first 5 rows of the dataframe

print (df.head())

                 Barangay         City  Latitude  Longitude
0          Addition Hills  Mandaluyong   14.5816   121.0384
1                  Alicia  Quezon City   14.6614   121.0253
2                  Amihan  Quezon City   14.6320   121.0680
3  Amoranto Senior Avenue  Quezon City   14.6300   121.0231
4         Apolonio Samson  Quezon City   14.6540   121.0090


In [83]:
address = 'Metro Manila, PH'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinates of Metro Manila are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Metro Manila are 14.5736108, 121.0329706.


In [84]:
map_Manila = folium.Map(location=[latitude, longitude], zoom_start=11)


for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Barangay']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Manila)  
    
map_Manila

In [85]:
CLIENT_ID = 'OWVSO0DY5DML30AC5K5EQMJ5COFM5S3SOX2EGPGG03TJD00V'
CLIENT_SECRET = 'HL1POL2PUYPLWBKAKWW2X2JNPQ3L4D3QDJN4LDO2H1O1Z0MB' 
VERSION = '20180605'
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OWVSO0DY5DML30AC5K5EQMJ5COFM5S3SOX2EGPGG03TJD00V
CLIENT_SECRET:HL1POL2PUYPLWBKAKWW2X2JNPQ3L4D3QDJN4LDO2H1O1Z0MB


In [86]:
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
 
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
    
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
       
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Barangay', 
                  'Barangay Latitude', 
                  'Barangay Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [87]:
Manila_venues = getNearbyVenues(names=df['Barangay'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Addition Hills
Alicia
Amihan
Amoranto Senior Avenue
Apolonio Samson
Aurora
Baesa
Bagbag
Bagong Lipunan ng Camp Crame
Bagong Pag-asa
Bagong Silang
Bagong Silangan
Bagumbayan
Bagumbuhay
Bahay Toro
Balingasa
Balintawak
Balong Bato
Barangka Drive
Barangka Ibaba
Barangka Ilaya
Barangka Itaas
Batasan Hills
Bayanihan
Blue Ridge A
Blue Ridge B
Botocan
Buayang Bato
Bungad
Burol
Camp Aguinaldo
Capri
Central
Claro
Commonwealth
Culiat
Daang Bakal
Damar
Damayan
Damayang Lagi
Del Monte
Dioquino Zobel
Don Manuel
Doña Imelda
Doña Josefa
Duyan Duyan
E. Rodriguez
East Kamias
Escopa I
Escopa II
Escopa III
Escopa IV
Fairview
Greater Lagro
Gulod
Hagdang Bato Itaas
Hagdang Bato Libis
Harapin ang Bukas
Highway Hills
Holy Spirit
Horseshoe
Hulo
Immaculate Concepcion
Kaligayahan
Kalusugan
Kamuning
Katipunan
Kaunlaran
Kristong Hari
Krus na Ligas
Laging Handa
Libis
Lourdes
Loyola Heights
Mabini-J. Rizal
Maharlika
Malamig
Malaya
Mandaluyong (Pob.)
Mangga
Manresa
Mariana
Mariblo
Marilag
Masagana
Masambong
Matandang

In [88]:
print(Manila_venues.shape)
Manila_venues.head()

(738, 7)


,Barangay,Barangay Latitude,Barangay Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Addition Hills,14.5816,121.0384,Worlwide Corporate Center Food Court,14.582606,121.038334,Food Court
1,Alicia,14.6614,121.0253,7-Eleven,14.660103,121.026284,Convenience Store
2,Alicia,14.6614,121.0253,7-Eleven,14.662062,121.025295,Convenience Store
3,Alicia,14.6614,121.0253,7-Eleven,14.660059,121.025606,Convenience Store
4,Amihan,14.6320,121.0680,Goolayan sa Batino,14.633251,121.069298,Convenience Store


In [89]:
Manila_venues.groupby('Barangay').count()

,Barangay Latitude,Barangay Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Barangay,,,,,,
Addition Hills,1,1,1,1,1,1
Alicia,3,3,3,3,3,3
Amihan,1,1,1,1,1,1
Amoranto Senior Avenue,8,8,8,8,8,8
Baesa,4,4,4,4,4,4
Bagbag,4,4,4,4,4,4
Bagong Lipunan ng Camp Crame,5,5,5,5,5,5
Bagong Silang,5,5,5,5,5,5
Bagumbayan,1,1,1,1,1,1


In [90]:
print('There are {} uniques categories.'.format(len(Manila_venues['Venue Category'].unique())))

There are 146 uniques categories.


In [91]:
Manila_onehot = pd.get_dummies(Manila_venues[['Venue Category']], prefix="", prefix_sep="")


Manila_onehot['Barangay'] = Manila_venues['Barangay'] 


fixed_columns = [Manila_onehot.columns[-1]] + list(Manila_onehot.columns[:-1])
Manila_onehot = Manila_onehot[fixed_columns]

Manila_onehot.head()

,Barangay,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Café,Campground,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gas Station,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Historic Site,Hobby Shop,Hotel,Hotel Pool,Ice Cream Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mongolian Restaurant,Mountain,Movie Theater,Music Store,Nail Salon,Nightclub,Noodle House,Optical Shop,Park,Pet Service,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Record Shop,Resort,Rest Area,Restaurant,Road,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shawarma Place,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Track Stadium,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Addition Hills,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alicia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alicia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alicia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Amihan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [92]:
Manila_onehot.shape

(738, 147)

In [93]:
Manila_grouped = Manila_onehot.groupby('Barangay').mean().reset_index()
Manila_grouped

,Barangay,American Restaurant,Arcade,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Café,Campground,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gas Station,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Historic Site,Hobby Shop,Hotel,Hotel Pool,Ice Cream Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mongolian Restaurant,Mountain,Movie Theater,Music Store,Nail Salon,Nightclub,Noodle House,Optical Shop,Park,Pet Service,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Record Shop,Resort,Rest Area,Restaurant,Road,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shawarma Place,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Track Stadium,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Addition Hills,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000
1,Alicia,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0000,0.000000,0.000000,0.00000

In [94]:
Manila_grouped.shape

(137, 147)

In [95]:
num_top_venues = 5

for hood in Manila_grouped['Barangay']:
    print("----"+hood+"----")
    temp = Manila_grouped[Manila_grouped['Barangay'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Addition Hills----
           venue  freq
0     Food Court   1.0
1      Rest Area   0.0
2  Movie Theater   0.0
3    Music Store   0.0
4     Nail Salon   0.0


----Alicia----
                 venue  freq
0    Convenience Store   1.0
1  American Restaurant   0.0
2            Pet Store   0.0
3           Nail Salon   0.0
4            Nightclub   0.0


----Amihan----
                 venue  freq
0    Convenience Store   1.0
1  American Restaurant   0.0
2            Pet Store   0.0
3           Nail Salon   0.0
4            Nightclub   0.0


----Amoranto Senior Avenue----
                  venue  freq
0               Stadium  0.12
1  Gym / Fitness Center  0.12
2                  Café  0.12
3    Athletics & Sports  0.12
4       Badminton Court  0.12


----Baesa----
               venue  freq
0           Pharmacy  0.50
1    Warehouse Store  0.25
2  Convenience Store  0.25
3        Pet Service  0.00
4        Music Store  0.00


----Bagbag----
                  venue  freq
0    Salon / Barber

                 venue  freq
0          Coffee Shop  0.10
1       Ice Cream Shop  0.06
2          Pizza Place  0.06
3                  Spa  0.06
4  American Restaurant  0.03


----Mabini-J. Rizal----
                     venue  freq
0        Food & Drink Shop  0.25
1     Fast Food Restaurant  0.25
2  Health & Beauty Service  0.25
3         Basketball Court  0.25
4      American Restaurant  0.00


----Maharlika----
                 venue  freq
0          Coffee Shop   1.0
1  American Restaurant   0.0
2            Pet Store   0.0
3           Nail Salon   0.0
4            Nightclub   0.0


----Malamig----
               venue  freq
0              Diner  0.17
1    Bubble Tea Shop  0.17
2  Electronics Store  0.17
3           Tea Room  0.17
4      Grocery Store  0.17


----Malaya----
                  venue  freq
0    Chinese Restaurant  0.50
1  Fast Food Restaurant  0.25
2           Beer Garden  0.25
3   American Restaurant  0.00
4           Pet Service  0.00


----Mandaluyong (Pob.)----
  

                 venue  freq
0        Grocery Store  0.25
1       Ice Cream Shop  0.25
2       Cosmetics Shop  0.25
3                  Spa  0.25
4  American Restaurant  0.00


----Socorro----
                  venue  freq
0           Pizza Place  0.22
1  Gym / Fitness Center  0.11
2  Fast Food Restaurant  0.11
3       Bubble Tea Shop  0.11
4        Sandwich Place  0.11


----South Triangle----
                 venue  freq
0                Diner  0.09
1                  Bar  0.09
2                  Spa  0.09
3        Burrito Place  0.05
4  Japanese Restaurant  0.05


----Tagumpay----
                  venue  freq
0  Fast Food Restaurant   0.4
1    Chinese Restaurant   0.2
2            Donut Shop   0.2
3           Pizza Place   0.2
4   American Restaurant   0.0


----Talayan----
                  venue  freq
0          Tennis Court  0.33
1  Fast Food Restaurant  0.33
2      Basketball Court  0.33
3   American Restaurant  0.00
4                  Park  0.00


----Talipapa----
             

In [96]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Barangay']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


Barangay_venues_sorted = pd.DataFrame(columns=columns)
Barangay_venues_sorted['Barangay'] = Manila_grouped['Barangay']

for ind in np.arange(Manila_grouped.shape[0]):
    Barangay_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Manila_grouped.iloc[ind, :], num_top_venues)

Barangay_venues_sorted.head()

,Barangay,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Addition Hills,Food Court,Deli / Bodega,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant,Electronics Store
1,Alicia,Convenience Store,Yoga Studio,Electronics Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant,Dumpling Restaurant
2,Amihan,Convenience Store,Yoga Studio,Electronics Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant,Dumpling Restaurant
3,Amoranto Senior Avenue,Filipino Restaurant,Plaza,Athletics & Sports,Badminton Court,Gym Pool,Gym / Fitness Center,Stadium,Café,Empanada Restaurant,Fast Food Restaurant
4,Baesa,Pharmacy,Warehouse Store,Convenience Store,Yoga Studio,Dumpling Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant


In [98]:
kclusters = 8

Manila_grouped_clustering = Manila_grouped.drop('Barangay', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Manila_grouped_clustering)


kmeans.labels_[0:10] 

array([7, 2, 2, 7, 7, 7, 7, 7, 6, 4], dtype=int32)

In [99]:
Barangay_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [100]:
Manila_merged = df

Manila_merged = Manila_merged.join(Barangay_venues_sorted.set_index('Barangay'), on='Barangay')

Manila_merged.head() 

,Barangay,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Addition Hills,Mandaluyong,14.5816,121.0384,7.0,Food Court,Deli / Bodega,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant,Electronics Store
1,Alicia,Quezon City,14.6614,121.0253,2.0,Convenience Store,Yoga Studio,Electronics Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant,Dumpling Restaurant
2,Amihan,Quezon City,14.6320,121.0680,2.0,Convenience Store,Yoga Studio,Electronics Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant,Dumpling Restaurant
3,Amoranto Senior Avenue,Quezon City,14.6300,121.0231,7.0,Filipino Restaurant,Plaza,Athletics & Sports,Badminton Court,Gym Pool,Gym / Fitness Center,Stadium,Café,Empanada Restaurant,Fast Food Restaurant
4,Apolonio Samson,Quezon City,14.6540,121.0090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
Manila_merged.dropna(inplace=True)
Manila_merged['Cluster Labels']=Manila_merged['Cluster Labels'].astype('int64') 
Manila_merged

,Barangay,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Addition Hills,Mandaluyong,14.58160,121.03840,7,Food Court,Deli / Bodega,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant,Electronics Store
1,Alicia,Quezon City,14.66140,121.02530,2,Convenience Store,Yoga Studio,Electronics Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant,Dumpling Restaurant
2,Amihan,Quezon City,14.63200,121.06800,2,Convenience Store,Yoga Studio,Electronics Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant,Dumpling Restaurant
3,Amoranto Senior Avenue,Quezon City,14.63000,121.02310,7,Filipino Restaurant,Plaza,Athletics & Sports,Badminton Court,Gym Pool,Gym / Fitness Center,Stadium,Café,Empanada Restaurant,Fast Food Restaurant
6,Baesa,Quezon City,14.67240,121.01130,7,Pharmacy,Warehouse Store,Convenience Store,Yoga Studio,Dumpling Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant
7,Bagbag,Quezon City,14.69660,121.03250,7,Bar,Fast Food Restaurant,Convenience Store,Salon / Barbershop,Dumpling Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant,Electronics Store
8,Bagong Lipunan ng Camp Crame,Quezon City,14.60694,121.05444,7,Chinese Restaurant,Entertainment Service,Spa,Bus Station,Gym,Yoga Studio,Farmers Market,English Restaurant,Empanada Restaurant,Electronics Store
10,Bagong Silang,Mandaluyong,14.59100,121.03140,7,Basketball Court,Chinese Restaurant,Japanese Restaurant,Fried Chicken Joint,Gas Station,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant
12,Bagumbayan,Quezon City,14.60540,121.08190,6,Café,Yoga Studio,Food & Drink Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant,Electronics Store
13,Bagumbuhay,Quezon City,14.61950,121.06700,4,Basketball Court,BBQ Joint,Yoga Studio,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Entertainment Service,English Restaurant,Empanada Restaurant


In [102]:

    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]


    markers_colors = []
    for lat, lon, poi, cluster in zip(Manila_merged['Latitude'], Manila_merged['Longitude'], Manila_merged['Barangay'], Manila_merged['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)

    map_clusters